In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

In [2]:
x = input("Enter a Stock Ticker")
df = yf.download(x)
today = datetime.today()
today = today.strftime("%Y %m %d")
#Make sure the the ranged entered is at least a year long
df = df.loc["2023-01-01":today].copy()
upper = 200
lower = 100

[*********************100%%**********************]  1 of 1 completed


In [3]:
df = df.drop(['Adj Close'], axis=1)
df['maLow'] = df.Close.rolling(lower).mean()
df['maHigh'] = df.Close.rolling(upper).mean()

In [4]:
df['ema_short'] = df['Close'].ewm(span=lower, adjust=False).mean()
df['ema_long'] = df['Close'].ewm(span=upper, adjust=False).mean()

In [5]:
def rating():

    SMAdistancePercent = df['maLow'].iloc[-1] / df['maHigh'].iloc[-1]
    SMAdistancePercentFactor = SMAdistancePercent / 1.15

    EMAdistancePercent = df['ema_short'].iloc[-1] / df['ema_long'].iloc[-1]
    EMAdistancePercentFactor = EMAdistancePercent / 1.1

    PriceTrend = 0
    if(df['Close'].iloc[-252] < df['Close'].iloc[-1]):
        PriceTrend += 0.25
    if(df['Close'].iloc[-126] < df['Close'].iloc[-1]):
        PriceTrend += 0.25
    if(df['Close'].iloc[-28] < df['Close'].iloc[-1]):
        PriceTrend += 0.25
    if(df['Close'].iloc[-5] < df['Close'].iloc[-1]):
        PriceTrend += 0.25


    dy = np.gradient(df['ema_short'])
    EMALowTrend = 0

    if(dy[-1] > dy[-5]):
        EMALowTrend += 0.4
    if(dy[-1] > dy[-28]):
        EMALowTrend += 0.3
    if(dy[-1] > dy[126]):
        EMALowTrend += 0.2
    if(dy[-1] > 0):
        EMALowTrend += 0.1



    dySMA = np.gradient(df['maLow'])
    SMALowTrend = 0

    if(dySMA[-1] > dySMA[-5]):
        SMALowTrend += 0.4
    if(dySMA[-1] > dySMA[-28]):
        SMALowTrend += 0.3
    if(dySMA[-1] > dySMA[126]):
        SMALowTrend += 0.2
    if(dySMA[-1] > 0):
        SMALowTrend += 0.1

    currentBelowShorter = 0
    if(df['Close'].iloc[-1] > df['maLow'].iloc[-1]):
        currentBelowShorter += 0.5

    if(df['Close'].iloc[-1] > df['ema_short'].iloc[-1]):
        currentBelowShorter += 0.5


    currentBelowLonger = 0
    if(df['Close'].iloc[-1] > df['maHigh'].iloc[-1]):
        currentBelowLonger += 0.5

    if(df['Close'].iloc[-1] > df['ema_long'].iloc[-1]):
        currentBelowLonger += 0.5

    FinalStockRating = SMAdistancePercentFactor * 0.2 + EMAdistancePercentFactor * 0.2 + PriceTrend * 0.2 + SMALowTrend * 0.1 + EMALowTrend * 0.1 + currentBelowShorter * 0.1 + currentBelowLonger * 0.1
    return FinalStockRating

buyRating = str(round(rating() * 100, 3)) + "%"

print(buyRating)

71.963%
